In [1]:
import os
os.environ["PYSPARK_PYTHON"] = 'python3' #"/usr/local/bin/python3" ".../usr/bin/python3"
'''/usr/local/lib/python3.9/dist-packages'''

'/usr/local/lib/python3.9/dist-packages'

In [2]:
from pyspark.sql import SparkSession

spark = SparkSession.\
        builder.\
        appName("HW4").\
        master("spark://spark-master:7077").\
        config("spark.executor.memory", "2g").\
        getOrCreate()

21/12/28 12:47:49 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [3]:
from io import BytesIO
from zipfile import ZipFile
from urllib.request import urlopen
# or: requests.get(url).content

resp = urlopen("https://files.grouplens.org/datasets/movielens/ml-1m.zip")
zipfile = ZipFile(BytesIO(resp.read()))
zipfile.namelist()

['ml-1m/',
 'ml-1m/movies.dat',
 'ml-1m/ratings.dat',
 'ml-1m/README',
 'ml-1m/users.dat']

In [4]:
#!pip install numpy
#!pip install pandas
import pandas as pd

header_list1 = ["UserID", "MovieID", "Rating", "Timestamp"]
header_list2 = ["MovieID", "Title", "Genres"]
header_list3 = ["UserID", "Gender", "Age", "Occupation", "Zip-code"]
pRatings = pd.read_csv (zipfile.open('ml-1m/ratings.dat'), sep='::', engine ='python', names=header_list1)
pMovies = pd.read_csv (zipfile.open('ml-1m/movies.dat'), sep='::', engine ='python', names=header_list2, encoding = "ISO-8859-1")
pUsers = pd.read_csv (zipfile.open('ml-1m/users.dat'), sep='::',engine ='python', names=header_list3)
pMovies

,MovieID,Title,Genres
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy
...,...,...,...
3878,3948,Meet the Parents (2000),Comedy
3879,3949,Requiem for a Dream (2000),Drama
3880,3950,Tigerland (2000),Drama
3881,3951,Two Family House (2000),Drama


In [5]:
#Pandas to spark dataframe

Ratings = spark.createDataFrame(pRatings)
Movies = spark.createDataFrame(pMovies)
Users = spark.createDataFrame(pUsers)
    
Movies.show()

+-------+--------------------+--------------------+
|MovieID|               Title|              Genres|
+-------+--------------------+--------------------+
|      1|    Toy Story (1995)|Animation|Childre...|
|      2|      Jumanji (1995)|Adventure|Childre...|
|      3|Grumpier Old Men ...|      Comedy|Romance|
|      4|Waiting to Exhale...|        Comedy|Drama|
|      5|Father of the Bri...|              Comedy|
|      6|         Heat (1995)|Action|Crime|Thri...|
|      7|      Sabrina (1995)|      Comedy|Romance|
|      8| Tom and Huck (1995)|Adventure|Children's|
|      9| Sudden Death (1995)|              Action|
|     10|    GoldenEye (1995)|Action|Adventure|...|
|     11|American Presiden...|Comedy|Drama|Romance|
|     12|Dracula: Dead and...|       Comedy|Horror|
|     13|        Balto (1995)|Animation|Children's|
|     14|        Nixon (1995)|               Drama|
|     15|Cutthroat Island ...|Action|Adventure|...|
|     16|       Casino (1995)|      Drama|Thriller|
|     17|Sen

#### Top-rated movies by all users

In [6]:
#Top-rated movies 
Ratings.groupBy("MovieID")\
    .agg({'Rating': 'avg'})\
    .sort('avg(Rating)', ascending=False)\
    .show(20, False)

21/12/28 12:48:52 WARN TaskSetManager: Stage 1 contains a task of very large size (3864 KiB). The maximum recommended task size is 1000 KiB.


+-------+-----------------+
|MovieID|avg(Rating)      |
+-------+-----------------+
|3656   |5.0              |
|3172   |5.0              |
|3881   |5.0              |
|1830   |5.0              |
|787    |5.0              |
|3382   |5.0              |
|3233   |5.0              |
|3280   |5.0              |
|989    |5.0              |
|3607   |5.0              |
|3245   |4.8              |
|53     |4.75             |
|2503   |4.666666666666667|
|2905   |4.608695652173913|
|2019   |4.560509554140127|
|318    |4.554557700942973|
|858    |4.524966261808367|
|745    |4.52054794520548 |
|50     |4.517106001121705|
|527    |4.510416666666667|
+-------+-----------------+
only showing top 20 rows



#### Top-rated movies grouped by gender, by age group, and by occupation, respectively

In [7]:
#Generating new dataframe ratings by user
from pyspark.sql.functions import col, udf, split, explode, first

Users = Users.withColumnRenamed("UserID", "UserID1")
userRatings = Ratings.join(Users,Ratings.UserID ==  Users.UserID1,"inner")\
    .drop('UserID1')
userRatings.show(truncate=False)

21/12/28 12:49:03 WARN TaskSetManager: Stage 3 contains a task of very large size (3864 KiB). The maximum recommended task size is 1000 KiB.


+------+-------+------+---------+------+---+----------+--------+
|UserID|MovieID|Rating|Timestamp|Gender|Age|Occupation|Zip-code|
+------+-------+------+---------+------+---+----------+--------+
|26    |3421   |3     |978141452|M     |25 |7         |23112   |
|26    |3422   |3     |978271825|M     |25 |7         |23112   |
|26    |1179   |4     |978274105|M     |25 |7         |23112   |
|26    |3423   |3     |978147079|M     |25 |7         |23112   |
|26    |719    |3     |978271742|M     |25 |7         |23112   |
|26    |2622   |3     |978275603|M     |25 |7         |23112   |
|26    |648    |3     |978139685|M     |25 |7         |23112   |
|26    |3426   |3     |978271666|M     |25 |7         |23112   |
|26    |2628   |2     |978143313|M     |25 |7         |23112   |
|26    |1682   |5     |978138796|M     |25 |7         |23112   |
|26    |3287   |2     |978140681|M     |25 |7         |23112   |
|26    |2558   |3     |978271539|M     |25 |7         |23112   |
|26    |216    |2     |97

In [8]:
#Top-rated movies grouped by gender, by age group, and by occupation, respectively
#movie, gender, score
userRatings.groupBy("MovieID", "Gender")\
    .agg({"Rating": "avg"})\
    .sort("avg(Rating)", ascending=False)\
    .show(20, False)

21/12/28 12:49:05 WARN TaskSetManager: Stage 6 contains a task of very large size (3864 KiB). The maximum recommended task size is 1000 KiB.


+-------+------+-----------+
|MovieID|Gender|avg(Rating)|
+-------+------+-----------+
|3280   |M     |5.0        |
|3817   |F     |5.0        |
|439    |M     |5.0        |
|3382   |F     |5.0        |
|3172   |M     |5.0        |
|2332   |F     |5.0        |
|1071   |F     |5.0        |
|1780   |F     |5.0        |
|1450   |F     |5.0        |
|989    |M     |5.0        |
|1420   |F     |5.0        |
|985    |M     |5.0        |
|3656   |M     |5.0        |
|3881   |F     |5.0        |
|3245   |F     |5.0        |
|53     |F     |5.0        |
|687    |F     |5.0        |
|2981   |F     |5.0        |
|2675   |F     |5.0        |
|3607   |F     |5.0        |
+-------+------+-----------+
only showing top 20 rows



In [9]:
#movie, age group, score
userRatings.groupBy("MovieID", "Age")\
    .agg({"Rating": "AVG"})\
    .sort("avg(Rating)", ascending=False)\
    .show(20, False)

21/12/28 12:49:17 WARN TaskSetManager: Stage 11 contains a task of very large size (3864 KiB). The maximum recommended task size is 1000 KiB.


+-------+---+-----------+
|MovieID|Age|avg(Rating)|
+-------+---+-----------+
|613    |1  |5.0        |
|1899   |56 |5.0        |
|3048   |18 |5.0        |
|3823   |56 |5.0        |
|3318   |1  |5.0        |
|38     |50 |5.0        |
|1067   |1  |5.0        |
|3092   |56 |5.0        |
|3640   |25 |5.0        |
|3192   |50 |5.0        |
|3532   |56 |5.0        |
|3781   |1  |5.0        |
|167    |25 |5.0        |
|570    |1  |5.0        |
|3464   |50 |5.0        |
|1809   |45 |5.0        |
|1002   |50 |5.0        |
|581    |1  |5.0        |
|1202   |1  |5.0        |
|1053   |50 |5.0        |
+-------+---+-----------+
only showing top 20 rows



In [10]:
#movie, occupation, score
userRatings.groupBy("MovieID", "occupation")\
    .agg({"Rating": "avg"})\
    .sort("avg(Rating)", ascending=False)\
    .show(20, False)

21/12/28 12:49:24 WARN TaskSetManager: Stage 15 contains a task of very large size (3864 KiB). The maximum recommended task size is 1000 KiB.


+-------+----------+-----------+
|MovieID|occupation|avg(Rating)|
+-------+----------+-----------+
|2249   |10        |5.0        |
|3741   |10        |5.0        |
|3311   |20        |5.0        |
|946    |18        |5.0        |
|2442   |5         |5.0        |
|1539   |20        |5.0        |
|567    |14        |5.0        |
|2345   |6         |5.0        |
|668    |5         |5.0        |
|1549   |0         |5.0        |
|2654   |10        |5.0        |
|1899   |2         |5.0        |
|1725   |13        |5.0        |
|2185   |12        |5.0        |
|279    |20        |5.0        |
|3038   |19        |5.0        |
|3925   |19        |5.0        |
|906    |10        |5.0        |
|967    |15        |5.0        |
|2776   |2         |5.0        |
+-------+----------+-----------+
only showing top 20 rows



#### Average rating score of each user for all movies, and grouped by genre, respectively

In [11]:
#Generating new dataframe movies user rating  
Movies = Movies.withColumnRenamed("MovieID", "MovieID1")
movieUserRatings = userRatings.join(Movies, userRatings.MovieID ==  Movies.MovieID1,"inner")\
    .drop('MovieID1')
movieUserRatings.show(truncate=False)

21/12/28 12:49:32 WARN TaskSetManager: Stage 19 contains a task of very large size (3864 KiB). The maximum recommended task size is 1000 KiB.


+------+-------+------+----------+------+---+----------+--------+--------------+------+
|UserID|MovieID|Rating|Timestamp |Gender|Age|Occupation|Zip-code|Title         |Genres|
+------+-------+------+----------+------+---+----------+--------+--------------+------+
|1447  |26     |5     |981019950 |M     |18 |4         |80521   |Othello (1995)|Drama |
|4139  |26     |4     |965350650 |M     |56 |14        |40207   |Othello (1995)|Drama |
|229   |26     |4     |1039503573|M     |18 |10        |04576   |Othello (1995)|Drama |
|3940  |26     |4     |965752989 |M     |35 |20        |32708   |Othello (1995)|Drama |
|1778  |26     |4     |975557687 |M     |18 |4         |94704   |Othello (1995)|Drama |
|3507  |26     |3     |966999088 |M     |25 |0         |02472   |Othello (1995)|Drama |
|5083  |26     |5     |962407158 |M     |25 |4         |16803   |Othello (1995)|Drama |
|4294  |26     |3     |1022309751|M     |1  |10        |75633   |Othello (1995)|Drama |
|3253  |26     |4     |968283674

In [12]:
#Average rating score of each user for all movies, and grouped by genre, respectively
#user, score
movieUserRatings.groupBy("UserID")\
    .agg({"Rating": "avg"})\
    .sort("avg(Rating)", ascending=False)\
    .show(20, False)

21/12/28 12:49:38 WARN TaskSetManager: Stage 23 contains a task of very large size (3864 KiB). The maximum recommended task size is 1000 KiB.


+------+------------------+
|UserID|avg(Rating)       |
+------+------------------+
|283   |4.962962962962963 |
|2339  |4.956521739130435 |
|3324  |4.904761904761905 |
|3902  |4.890909090909091 |
|446   |4.8431372549019605|
|447   |4.837837837837838 |
|4649  |4.818181818181818 |
|4634  |4.813725490196078 |
|1131  |4.796116504854369 |
|4925  |4.761904761904762 |
|4755  |4.76              |
|5069  |4.76              |
|4801  |4.73469387755102  |
|682   |4.733333333333333 |
|1670  |4.714285714285714 |
|91    |4.704545454545454 |
|3461  |4.702702702702703 |
|5768  |4.702702702702703 |
|5984  |4.7               |
|1021  |4.694656488549619 |
+------+------------------+
only showing top 20 rows



In [13]:
#user, genre, score
movieUserRatings.groupBy("UserID", "Genres")\
    .agg({"Rating": "avg"})\
    .sort("avg(Rating)", ascending=False)\
    .show(20, False)

21/12/28 12:49:51 WARN TaskSetManager: Stage 29 contains a task of very large size (3864 KiB). The maximum recommended task size is 1000 KiB.


+------+-----------------------------------+-----------+
|UserID|Genres                             |avg(Rating)|
+------+-----------------------------------+-----------+
|3916  |Sci-Fi                             |5.0        |
|137   |Adventure|Comedy|Sci-Fi            |5.0        |
|1601  |Action|Horror|Sci-Fi               |5.0        |
|4083  |Action|Adventure|Romance|Sci-Fi|War|5.0        |
|3513  |Horror                             |5.0        |
|5063  |Action|Adventure|Romance|Sci-Fi|War|5.0        |
|5270  |Comedy|Crime|Fantasy               |5.0        |
|1875  |Action|Adventure|Romance|Sci-Fi|War|5.0        |
|3292  |Action|Drama|Romance               |5.0        |
|3140  |Action|Adventure|Romance|Sci-Fi|War|5.0        |
|4832  |Film-Noir|Sci-Fi                   |5.0        |
|2454  |Action|Adventure|Romance|Sci-Fi|War|5.0        |
|3721  |Crime|Thriller                     |5.0        |
|4245  |Action|Adventure|Romance|Sci-Fi|War|5.0        |
|3147  |Action|Adventure|Romanc

#### Top-’similar’ users based on the cosine similarity of previous ratings each user has given. Sorted in descending order of ‘user’ similarity score.

In [14]:
from pyspark.sql import functions as F
userBased1 = movieUserRatings.groupby("UserID")\
                              .agg(F.collect_list("Rating").alias("Ratings1"))

userBased2 = movieUserRatings.groupby("UserID")\
                              .agg(F.collect_list("Rating").alias("Ratings2"))
userBased2.show()

21/12/28 12:50:02 WARN TaskSetManager: Stage 35 contains a task of very large size (3864 KiB). The maximum recommended task size is 1000 KiB.


+------+--------------------+
|UserID|            Ratings2|
+------+--------------------+
|    26|[4, 2, 3, 2, 3, 3...|
|    29|[2, 5, 3, 2, 4, 3...|
|   474|[4, 5, 5, 1, 3, 5...|
|  1697|[5, 4, 4, 4, 4, 4...|
|  1950|[3, 2, 3, 5, 3, 3...|
|  2040|[4, 1, 1, 3, 5, 5...|
|  2453|[5, 4, 4, 3, 2, 2...|
|  2509|[2, 1, 4, 4, 4, 3...|
|  2529|[4, 4, 4, 3, 3, 5...|
|  4823|[4, 4, 4, 4, 3, 3...|
|  1677|[5, 4, 1, 2, 4, 4...|
|  1806|[4, 4, 5, 4, 5, 4...|
|  2250|[4, 5, 2, 5, 1, 4...|
|  2927|[4, 3, 4, 3, 5, 3...|
|  3506|[4, 5, 4, 5, 5, 3...|
|  3764|[3, 5, 5, 2, 5, 4...|
|  4590|[3, 2, 4, 5, 3, 5...|
|  5556|[4, 3, 2, 3, 2, 3...|
|  3091|[5, 1, 4, 3, 4, 4...|
|   964|[4, 3, 3, 4, 3, 4...|
+------+--------------------+
only showing top 20 rows



In [15]:
userBased3 = movieUserRatings.select("UserID", "Age", "Occupation", "Rating")\
                            .groupby("UserID","Age", "Occupation")\
                            .agg({"Rating": "avg"})\
                            .sort("avg(Rating)", ascending=False)
userBased3.show()

21/12/28 12:50:13 WARN TaskSetManager: Stage 42 contains a task of very large size (3864 KiB). The maximum recommended task size is 1000 KiB.


+------+---+----------+------------------+
|UserID|Age|Occupation|       avg(Rating)|
+------+---+----------+------------------+
|   283| 25|         0| 4.962962962962963|
|  2339| 56|         3| 4.956521739130435|
|  3324| 35|        17| 4.904761904761905|
|  3902| 25|         5| 4.890909090909091|
|   446| 50|         0|4.8431372549019605|
|   447| 45|        11| 4.837837837837838|
|  4649| 56|        13| 4.818181818181818|
|  4634| 50|        20| 4.813725490196078|
|  1131| 56|        13| 4.796116504854369|
|  4925| 35|         6| 4.761904761904762|
|  5069| 56|         0|              4.76|
|  4755| 56|         7|              4.76|
|  4801| 25|         4|  4.73469387755102|
|   682| 25|         4| 4.733333333333333|
|  1670| 56|        13| 4.714285714285714|
|    91| 35|         7| 4.704545454545454|
|  3461| 35|         0| 4.702702702702703|
|  5768|  1|         7| 4.702702702702703|
|  5984| 25|         1|               4.7|
|  1021| 35|         0| 4.694656488549619|
+------+---

In [16]:
userBased1 = userBased1.withColumnRenamed("UserID", "UserID1")
userBased2 = userBased2.withColumnRenamed("UserID", "UserID2")

size = userBased1.limit(1).select(F.size('Ratings1')).first()[0]
crossUser = userBased1.crossJoin(userBased2)

# Get cosine similarity
userResult = crossUser.select('UserID1','UserID2',
                       sum([F.col('Ratings1')[i] * F.col('Ratings2')[i] for i in range(size)]).alias('dot_product'),
                       F.sqrt(sum([F.col('Ratings1')[i] * F.col('Ratings1')[i] for i in range(size)])).alias('norm1'),
                       F.sqrt(sum([F.col('Ratings2')[i] * F.col('Ratings2')[i] for i in range(size)])).alias('norm2'))\
                        .selectExpr('UserID1', 'UserID2', 'dot_product / norm1 / norm2 as cosine_similarity')
userResult.show()

21/12/28 12:50:24 WARN TaskSetManager: Stage 48 contains a task of very large size (3864 KiB). The maximum recommended task size is 1000 KiB.
21/12/28 12:50:38 WARN TaskSetManager: Stage 54 contains a task of very large size (3864 KiB). The maximum recommended task size is 1000 KiB.


+-------+-------+------------------+
|UserID1|UserID2| cosine_similarity|
+-------+-------+------------------+
|     26|     26|               1.0|
|     26|     29|0.9159281935970004|
|     26|   1697|0.9413624426947305|
|     26|   1950|0.9073710145623568|
|     26|   2040|              null|
|     26|   2453|0.9086757824609223|
|     26|   2529|0.9170126568304348|
|     26|   3091|0.9282404191423106|
|     26|   3506|0.9286935505748676|
|     26|   4823|0.8750061515700919|
|     26|   5556|0.9323289920821245|
|     26|    474|0.8954791004116868|
|     26|    964|0.9328019735787175|
|     26|   1806|0.9223192159494613|
|     26|   4894|              null|
|     26|   1677|              null|
|     26|   5385|              null|
|     26|   2509|              null|
|     26|   2214| 0.936572670617149|
|     26|   2927|0.9142506694329033|
+-------+-------+------------------+
only showing top 20 rows



#### Top-’similar’ movies based on the cosine similarity of previous ratings each movie received. Sorted in descending order of ‘item’ similarity score.

In [20]:
movieBased1 = movieUserRatings.groupby("MovieID")\
                              .agg(F.collect_list("Rating").alias("Ratings1"))
movieBased2 = movieUserRatings.groupby("MovieID")\
                              .agg(F.collect_list("Rating").alias("Ratings2"))
movieBased2.show()

21/12/28 12:53:00 WARN TaskSetManager: Stage 85 contains a task of very large size (3864 KiB). The maximum recommended task size is 1000 KiB.


+-------+--------------------+
|MovieID|            Ratings2|
+-------+--------------------+
|     26|[4, 4, 2, 4, 5, 4...|
|     29|[4, 5, 4, 4, 5, 5...|
|    474|[4, 5, 4, 4, 4, 4...|
|    964|[5, 2, 3, 3, 3, 4...|
|   1677|[2, 3, 3, 3, 3, 4...|
|   1806|[2, 1, 3, 5, 2, 3...|
|   1950|[4, 5, 5, 5, 5, 5...|
|   2040|[3, 2, 2, 1, 3, 3...|
|   2214|                 [3]|
|   2250|[3, 4, 5, 3, 3, 4...|
|   2453|[2, 1, 3, 4, 4, 3...|
|   2509|[2, 4, 3, 2, 2, 2...|
|   2529|[5, 5, 4, 4, 5, 1...|
|   2927|[5, 3, 4, 5, 5, 5...|
|   3091|[4, 5, 4, 3, 4, 5...|
|   3506|[4, 3, 3, 3, 4, 3...|
|   3764|[3, 1, 3, 2, 2, 1...|
|     65|[1, 4, 1, 1, 3, 1...|
|    191|[3, 2, 2, 2, 2, 3...|
|    418|[5, 4, 2, 1, 2, 2...|
+-------+--------------------+
only showing top 20 rows



In [22]:
movieBased1 = movieBased1.withColumnRenamed("MovieID", "MovieID1")
movieBased2 = movieBased2.withColumnRenamed("MovieID", "MovieID2")

size = movieBased1.limit(1).select(F.size('Ratings1')).first()[0]
crossMovie = movieBased1.crossJoin(movieBased2)

# Get cosine similarity
movieResult = crossMovie.select('MovieID1','MovieID2',
                       sum([F.col('Ratings1')[i] * F.col('Ratings2')[i] for i in range(size)]).alias('dot_product'),
                       F.sqrt(sum([F.col('Ratings1')[i] * F.col('Ratings1')[i] for i in range(size)])).alias('norm1'),
                       F.sqrt(sum([F.col('Ratings2')[i] * F.col('Ratings2')[i] for i in range(size)])).alias('norm2'))\
                        .selectExpr('MovieID1', 'MovieID2', 'dot_product / norm1 / norm2 as cosine_similarity')
movieResult.show()

21/12/28 13:08:43 WARN TaskSetManager: Stage 105 contains a task of very large size (3864 KiB). The maximum recommended task size is 1000 KiB.
21/12/28 13:08:56 ERROR CodeGenerator: failed to compile: org.codehaus.janino.InternalCompilerException: Compiling "GeneratedClass" in "generated.java": Code of method "processNext()V" of class "org.apache.spark.sql.catalyst.expressions.GeneratedClass$GeneratedIteratorForCodegenStage21" grows beyond 64 KB
org.codehaus.janino.InternalCompilerException: Compiling "GeneratedClass" in "generated.java": Code of method "processNext()V" of class "org.apache.spark.sql.catalyst.expressions.GeneratedClass$GeneratedIteratorForCodegenStage21" grows beyond 64 KB
	at org.codehaus.janino.UnitCompiler.compile2(UnitCompiler.java:366)
	at org.codehaus.janino.UnitCompiler.access$000(UnitCompiler.java:226)
	at org.codehaus.janino.UnitCompiler$1.visitCompilationUnit(UnitCompiler.java:336)
	at org.codehaus.janino.UnitCompiler$1.visitCompilationUnit(UnitCompiler.java:

+--------+--------+------------------+
|MovieID1|MovieID2| cosine_similarity|
+--------+--------+------------------+
|      26|      26|1.0000000000000002|
|      26|      29|0.9326760467251204|
|      26|     474|0.9373893041819876|
|      26|     964|              null|
|      26|    1677|              null|
|      26|    1806|0.8944765914657672|
|      26|    1950| 0.954862337256213|
|      26|    2040| 0.920760943037605|
|      26|    2214|              null|
|      26|    2250|              null|
|      26|    2453|0.9114702178469216|
|      26|    2509|              null|
|      26|    2529|0.9276966087822656|
|      26|    2927|              null|
|      26|    3091|0.9467018069193637|
|      26|    3506|0.9313570792579194|
|      26|    3764|0.9086157403590003|
|      29|      26|0.9326760467251203|
|      29|      29|1.0000000000000002|
|      29|     474|0.9475759647269894|
+--------+--------+------------------+
only showing top 20 rows

